In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.family'] = 'IPAPGothic' #全体のフォントを設定

%matplotlib inline

# データセットの作成

In [2]:
R_arr = np.array([
    [1, 3, np.nan,  3],
    [np.nan, 1, 3, np.nan],
    [2, 1, 3, 1],
    [1, 3, 2, np.nan]
])

R_df = pd.DataFrame(
    R_arr, 
    columns=['親子丼', '牛丼', '海鮮丼', 'カツ丼'], 
    index=['山田', '田中', '佐藤', '鈴木']
)
R_df.head()

,親子丼,牛丼,海鮮丼,カツ丼
山田,1.0,3.0,NaN,3.0
田中,NaN,1.0,3.0,NaN
佐藤,2.0,1.0,3.0,1.0
鈴木,1.0,3.0,2.0,NaN


上記のデータフレームを以下の形式に変換します。

| ユーザ | アイテム | 評価値 |
|:------:|:--------:|:------:|
| 山田 | 親子丼 | 1.0 | 
| 山田 | 牛丼 | 3.0 |
| 山田 | 海鮮丼 | NaN |
| ... | ... | ... |
| 鈴木 | 海鮮丼 | 2.0 |
| 鈴木 | カツ丼 | NaN |

In [3]:
R_df = R_df.stack().reset_index().rename(columns={'level_0': 'user', 'level_1': 'item', 0: 'rating'})
R_df.head()

,user,item,rating
0,山田,親子丼,1.0
1,山田,牛丼,3.0
2,山田,カツ丼,3.0
3,田中,牛丼,1.0
4,田中,海鮮丼,3.0


In [8]:
from surprise import Reader, Dataset

reader = Reader(line_format='user item rating', sep=' ')
dataset = Dataset.load_from_df(R_df, reader)

In [24]:
from surprise import KNNBasic

trainset = dataset.build_full_trainset()

sim_options = {
    'name': 'pearson',   # 類似度を計算する方法を指定（ cosine,msd,pearson,pearson_baseline ）
    'user_based': True # False にするとアイテムベースに
}
model = KNNBasic(k=1, min_k=1,sim_options=sim_options)
model.fit(trainset)

Computing the pearson similarity matrix...
Done computing similarity matrix.


In [25]:
# 評価値を予測する
prediction = model.predict(uid="田中", iid="親子丼")

print('Predicted rating(User: {0}, Item: {1}): {2:.2f}'.format(prediction.uid, prediction.iid, prediction.est))

Predicted rating(User: 田中, Item: 親子丼): 2.00
